In [1]:
import os
%pwd

'e:\\Code\\NLP_Text_Summarize_Project\\research'

In [2]:
os.chdir("../")
%pwd

'e:\\Code\\NLP_Text_Summarize_Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list

In [4]:
from src.Text_Summarize.constants import *
from src.Text_Summarize.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES
        )

        return data_validation_config

In [6]:
from src.Text_Summarize import logger
import re
import langdetect
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        #... (your existing code)

    def validate_text_normalization(self) -> bool:
        for file in self.config.ALL_REQUIRED_FILES:
            with open(os.path.join("artifacts", "data_ingestion", "corpus", file), 'r') as f:
                text = f.read()
                if not self.is_normalized(text):
                    return False
        return True

    def is_normalized(self, text: str) -> bool:
        # Implement text normalization checks (e.g., lowercase, no special characters)
        pass

    def validate_tokenization(self) -> bool:
        for file in self.config.ALL_REQUIRED_FILES:
            with open(os.path.join("artifacts", "data_ingestion", "corpus", file), 'r') as f:
                text = f.read()
                if not self.is_tokenized(text):
                    return False
        return True

    def is_tokenized(self, text: str) -> bool:
        # Implement tokenization checks (e.g., words separated correctly)
        pass

    #... (add more validation methods for each step)

    def validate_all(self) -> bool:
        validation_status = True
        validation_status &= self.validate_all_files_exist()
        validation_status &= self.validate_text_normalization()
        validation_status &= self.validate_tokenization()
        #... (add more validation methods)
        return validation_status

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","data_ingestion","corpus"))

            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
            
            return validation_status
        
        except Exception as e:
            raise e

    def validate_text_normalization(self, text):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"[^\w\s]", "", text)  # Remove special characters and punctuation
        text = re.sub(r"\s+", " ", text)  # Normalize whitespace characters
        return text

    def validate_tokenization(self, text):
        tokens = word_tokenize(text)
        if len(tokens) < 2:  # Check for minimum token length
            return False
        return True

    def validate_language_detection(self, text):
        try:
            lang = langdetect.detect(text)
            return lang
        except langdetect.lang_detect_exception.LangDetectException:
            return "unknown"

    def validate_text_length(self, text):
        if len(text) < 10:  # Check for minimum text length
            return False
        return True

    def validate_stopwords(self, text):
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words("english"))
        if any(token in stop_words for token in tokens):  # Check for stopwords
            return False
        return True

    def validate_all(self) -> bool:
        validation_status = True

        for file in self.config.ALL_REQUIRED_FILES:
            with open(os.path.join("artifacts", "data_ingestion", "corpus", file), 'r') as f:
                text = f.read()
                text = self.validate_text_normalization(text)
                validation_status &= self.validate_tokenization(text)
                validation_status &= self.validate_language_detection(text)!= "unknown"
                validation_status &= self.validate_text_length(text)
                validation_status &= self.validate_stopwords(text)

        return validation_status

In [7]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            all_files = os.listdir(os.path.join("artifacts", "data_ingestion", "corpus"))
            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write("Validation status: validate_all_files_exist - False\n")
                    return False
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write("Validation status: validate_all_files_exist - True\n")
            return True
        except Exception as e:
            raise e

    def validate_text_normalization(self, text) -> str:
        try:
            text = text.lower()  # Convert to lowercase
            text = re.sub(r"[^\w\s]", "", text)  # Remove special characters and punctuation
            text = re.sub(r"\s+", " ", text)  # Normalize whitespace characters
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write("Validation status: validate_text_normalization - True\n")
            return text
        except Exception as e:
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Validation status: validate_text_normalization - False ({str(e)})\n")
            return ""

    def validate_tokenization(self, text) -> bool:
        try:
            tokens = word_tokenize(text)
            if len(tokens) < 2:  # Check for minimum token length
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write("Validation status: validate_tokenization - False\n")
                return False
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write("Validation status: validate_tokenization - True\n")
            return True
        except Exception as e:
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Validation status: validate_tokenization - False ({str(e)})\n")
            return False

    def validate_language_detection(self, text) -> str:
        try:
            lang = langdetect.detect(text)
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Validation status: validate_language_detection - {lang}\n")
            return lang
        except langdetect.lang_detect_exception.LangDetectException:
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write("Validation status: validate_language_detection - unknown\n")
            return "unknown"

    def validate_text_length(self, text) -> bool:
        try:
            if len(text) < 10:  # Check for minimum text length
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write("Validation status: validate_text_length - False\n")
                return False
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write("Validation status: validate_text_length - True\n")
            return True
        except Exception as e:
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Validation status: validate_text_length - False ({str(e)})\n")
            return False

    def validate_stopwords(self, text) -> bool:
        try:
            tokens = word_tokenize(text)
            stop_words = set(stopwords.words("english"))
            if any(token in stop_words for token in tokens):  # Check for stopwords
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write("Validation status: validate_stopwords - False\n")
                return False
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write("Validation status: validate_stopwords - True\n")
            return True
        except Exception as e:
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Validation status: validate_stopwords - False ({str(e)})\n")
            return False

    def validate_all(self) -> bool:
        validation_status = True

        for file in self.config.ALL_REQUIRED_FILES:
            with open(os.path.join("artifacts", "data_ingestion", "corpus", file), 'r') as f:
                text = f.read()
                text = self.validate_text_normalization(text)
                if not self.validate_tokenization(text):
                    validation_status = False
                if self.validate_language_detection(text) == "unknown":
                    validation_status = False
                if not self.validate_text_length(text):
                    validation_status = False
                if not self.validate_stopwords(text):
                    validation_status = False

        return validation_status

In [8]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
    data_validation.validate_all()
except Exception as e:
    raise e

[2024-05-18 14:12:10,642: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2024-05-18 14:12:10,646: INFO: common: yaml file: params.yaml loaded succesfully]
[2024-05-18 14:12:10,649: INFO: common: created directory at: artifacts]
[2024-05-18 14:12:10,651: INFO: common: created directory at: artifacts/data_validation]
